In [1]:
import os
import numpy as np
from scipy.io import loadmat
import random
import time
import itertools
import cv2 as cv
from keras.models import load_model

In [2]:
##########################################################################
#TO COMPLETE 
original_path = ""
imgPath     = original_path + "TRAINING/"
##########################################################################

model_path = 'modelV85.h5'

IMAGE_SIZE = 180
colored = True
img_channels = 3 if colored else 1

if original_path=="":
  raise Exception("you should complete the original_path")



In [3]:
# model_path = 'model.h5'
model = load_model(model_path)

total_params = model.count_params()
print("The model contain ",total_params, " parameters")


# Get the size in MB
size_model = os.path.getsize(model_path)/ (1024*1024)  
print(f"Size of the model is : {size_model:.2f} MB")

# # Summary to check total params and ensure it's under 1 million
# my_FRmodel.summary()



The model contain  707473  parameters
Size of the model is : 8.17 MB


In [4]:
def CHALL_AGC_ComputeRecognScores(auto_ids, true_ids):
    #   Compute face recognition score
    #
    #   INPUTS
    #     - AutomSTR: The results of the automatic face
    #     recognition algorithm, stored as an integer
    #
    #     - AGC_Challenge_STR: The ground truth ids
    #
    #   OUTPUT
    #     - FR_score:     The final recognition score
    #
    #   --------------------------------------------------------------------
    #   AGC Challenge
    #   Universitat Pompeu Fabra
    #

    if len(auto_ids) != len(true_ids):
        assert ('Inputs must be of the same len');

    f_beta = 1
    res_list = list(filter(lambda x: true_ids[x] != -1, range(len(true_ids))))

    nTP = len([i for i in res_list if auto_ids[i] == true_ids[i]])

    res_list = list(filter(lambda x: auto_ids[x] != -1, range(len(auto_ids))))

    nFP = len([i for i in res_list if auto_ids[i] != true_ids[i]])

    res_list_auto_ids = list(filter(lambda x: auto_ids[x] == -1, range(len(auto_ids))))
    res_list_true_ids = list(filter(lambda x: true_ids[x] != -1, range(len(true_ids))))

    nFN = len(set(res_list_auto_ids).intersection(res_list_true_ids))

    FR_score = (1 + f_beta ** 2) * nTP / ((1 + f_beta ** 2) * nTP + f_beta ** 2 * nFN + nFP)

    return FR_score,nTP,res_list
    

In [5]:
#-- Face detection
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_alt.xml')

#parameter for the detection
scaleFactor  = 1.05
minNeighbors = 6
minsize      = 100
# maxsize      = 700

#I have to change the minsize for new images because they are 
def MyFaceDetectionFunction(img): #from lab 1
    frame_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    frame_gray = cv.equalizeHist(frame_gray)

    #-- Detect faces
    faces = face_cascade.detectMultiScale(
        frame_gray, 
        scaleFactor  = scaleFactor, 
        minNeighbors = minNeighbors, 
        minSize      = (minsize, minsize),
        # maxSize      = (maxsize, maxsize)
    )
    return faces

def select_two_biggest_faces(faces):
    areas = []
    M =[]
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x1 + w 
        y2 = y1 + h
        M.append([x1,y1,x2,y2])
        areas.append(w*h)
        
    if len(M)>2:
        first_face = 0
        second_face = 1
        if areas[first_face]<areas[second_face]: #check if the biggest face is on index 2, then swap index
            first_face, second_face = 1, 0
        for j in range(2,len(M)):
            if areas[j]>areas[first_face]:
                second_face = first_face
                first_face  = j
            elif areas[j]> areas[second_face]:
                second_face = j
        M = [M[first_face],M[second_face]]
    return M
  
#Preprocessing of the Data
def process_image(img):
    img_resized = cv.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    if not colored:
        grey_img = cv.cvtColor(img_resized, cv.COLOR_BGR2GRAY)
        face_normalized = grey_img /255.0
        face = np.expand_dims(face_normalized, axis=-1) 
        return face

    else:
        colored_img = cv.cvtColor(img_resized, cv.COLOR_BGR2RGB)
        face_normalized = colored_img /255.0

        return face_normalized
    
def crop_and_normalize_image(img,faces):
    cropped_and_normalized_faces =[]
    for (x1, y1, x2, y2) in faces:
            face = img[y1:y2, x1:x2]
            processed_image = process_image(face)
            cropped_and_normalized_faces.append(processed_image)
            # cv.imwrite(Saving_folder + im, img_resized)
    return cropped_and_normalized_faces

def detect_crop_and_normalize_image(img):
    # We apply face detection
    facesDetected = MyFaceDetectionFunction(img)
    cropped_and_normalized_faces=[]
    # We select the two biggest faces
    faces = select_two_biggest_faces(facesDetected)    
    have_a_face = len(faces)>0
    # Save_detection(img,faces,im)
    if have_a_face:
        #-- Handle images with detected faces
        have_a_face = True
        # Crop, normalize, and collect faces if detected
        cropped_and_normalized_faces = crop_and_normalize_image(img,faces)
    return cropped_and_normalized_faces,have_a_face


In [6]:
def predict(model, image):
    cropped_and_normalized_faces,have_a_face = detect_crop_and_normalize_image(image)
  
    if not have_a_face:
        return -1
    
    best_prediction = 0
    class_ = 0
    
    for face in cropped_and_normalized_faces:
        face_batch = np.expand_dims(face, axis=0)  # format the model to (1, 224, 224, 1)
        predictions = model.predict(face_batch,verbose=0)
        
        predicted_class = np.argmax(predictions, axis=1)
        val_prediction = np.max(predictions[0])
        
        if  val_prediction>best_prediction:
            best_prediction = predicted_class[0]
            class_ = predicted_class[0]
    if class_ ==0: 
        return -1
    else:
        return class_

In [7]:
# Basic script for Face Recognition Challenge
# --------------------------------------------------------------------
# AGC Challenge
# Universitat Pompeu Fabra

# Load challenge Training data
dir_challenge3 = original_path
AGC_Challenge3_TRAINING = loadmat(dir_challenge3 + "AGC_Challenge3_Training.mat")
AGC_Challenge3_TRAINING = np.squeeze(AGC_Challenge3_TRAINING['AGC_Challenge3_TRAINING'])

imageName = AGC_Challenge3_TRAINING['imageName']
imageName = list(itertools.chain.from_iterable(imageName))

ids = list(AGC_Challenge3_TRAINING['id'])
ids = np.concatenate(ids).ravel().tolist()

faceBox = AGC_Challenge3_TRAINING['faceBox']
faceBox = list(itertools.chain.from_iterable(faceBox))
    

# Initialize results structure
AutoRecognSTR = []

# Initialize timer accumulator
total_time = 0

# Load your FRModel
my_FRmodel = model

for idx, im in enumerate(imageName):

    image = cv.imread(imgPath + im)
        

    try:
        print(f"Image number : {idx}", end='\r')  
        ti = time.time()
        # Timer on
        ###############################################################
        # Your face recognition function goes here.It must accept 2 input parameters:

        # 1. the input image A
        # 2. the recognition model

        # and must return a single integer number as output, which can be:

        # a) A number between 1 and 80 (representing one of the identities in the training set)
        # b) A "-1" indicating that none of the 80 users is present in the input image
        
        autom_id = predict(my_FRmodel,image)
        tt = time.time() - ti
        total_time = total_time + tt
    except:
        # If the face recognition function fails, it will be assumed that no user was detected for his input image
        autom_id = random.randint(-1, 80)

    AutoRecognSTR.append(autom_id)

FR_score ,nTP, res_list= CHALL_AGC_ComputeRecognScores(AutoRecognSTR, ids)
_, rem = divmod(total_time, 3600)
minutes, seconds = divmod(rem, 60)
print('F1-score: %.2f, Total time: %2d m %.2f s' % (100 * FR_score, int(minutes), seconds))


F1-score: 85.22, Total time:  1 m 32.02 s
